# РК2
## Студент: вань хао
## Группа: ИУ5-24М
## Тема: Методы обработки текстов
## Решение задачи классификации текстов
### Необходимо решить задачу классификации текстов на основе любого выбранного Вами датасета (кроме примера, который рассматривался в лекции). Классификация ### может быть бинарной или многоклассовой. Целевой признак из выбранного Вами датасета может иметь любой физический смысл, примером является задача анализа ### тональности текста.

### Необходимо сформировать два варианта векторизации признаков - на основе CountVectorizer и на основе TfidfVectorizer.

### В качестве классификаторов необходимо использовать два классификатора по варианту для группы ИУ5-24М:

### Классификатор №1: KNeighborsClassifier
### Классификатор №2: Complement Naive Bayes - CNB

In [80]:
import os
import gzip
import shutil

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import ComplementNB

import warnings
warnings.filterwarnings('ignore')

In [81]:
df = pd.read_csv('/Users/wanhao/Downloads/毛子2/ммо/Tweets.csv')
df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [82]:
df=df.drop(['text','textID'],axis=1)

In [83]:
df.head()
df.dropna(axis=1,how='all')

,selected_text,sentiment
0,"I`d have responded, if I were going",neutral
1,Sooo SAD,negative
2,bullying me,negative
3,leave me alone,negative
4,"Sons of ****,",negative
...,...,...
27476,d lost,negative
27477,", don`t force",negative
27478,Yay good for both of you.,positive
27479,But it was worth it ****.,positive


In [84]:
tfidfv = TfidfVectorizer()
tfidf_ngram_features = tfidfv.fit_transform(df['selected_text'].values.astype('U'))
tfidf_ngram_features

<27481x17620 sparse matrix of type '<class 'numpy.float64'>'
	with 173222 stored elements in Compressed Sparse Row format>

In [85]:
countvec = CountVectorizer()
countvec_ngram_features = countvec.fit_transform(df['selected_text'].values.astype('U'))
countvec_ngram_features

<27481x17620 sparse matrix of type '<class 'numpy.int64'>'
	with 173222 stored elements in Compressed Sparse Row format>

In [86]:
# TFIDF + KNC
X_train, X_test, y_train, y_test = train_test_split(tfidf_ngram_features, df['sentiment'], test_size=0.3, random_state=1)
model = KNeighborsClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, digits=4, target_names=list(map(str, list(y_test.unique())))))

              precision    recall  f1-score   support

    negative     0.3553    0.9737    0.5206      2282
     neutral     0.7849    0.0430    0.0816      3392
    positive     0.9380    0.6585    0.7738      2571

    accuracy                         0.4925      8245
   macro avg     0.6927    0.5584    0.4587      8245
weighted avg     0.7137    0.4925    0.4189      8245



In [87]:

# CountVec + KNC
X_train, X_test, y_train, y_test = train_test_split(countvec_ngram_features, df['sentiment'], 
                                                    test_size=0.3, random_state=1)
model = KNeighborsClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, digits=4, target_names=list(map(str, list(y_test.unique())))))

              precision    recall  f1-score   support

    negative     0.4877    0.9032    0.6334      2282
     neutral     0.8755    0.2966    0.4431      3392
    positive     0.7087    0.7911    0.7477      2571

    accuracy                         0.6187      8245
   macro avg     0.6906    0.6636    0.6080      8245
weighted avg     0.7162    0.6187    0.5907      8245



In [88]:
# TFIDF + CNB
X_train, X_test, y_train, y_test = train_test_split(tfidf_ngram_features, df['sentiment'], test_size=0.3, random_state=1)
model = ComplementNB()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, digits=4, target_names=list(map(str, list(y_test.unique())))))

              precision    recall  f1-score   support

    negative     0.7994    0.7195    0.7574      2282
     neutral     0.7664    0.8723    0.8159      3392
    positive     0.8721    0.7904    0.8292      2571

    accuracy                         0.8045      8245
   macro avg     0.8126    0.7941    0.8008      8245
weighted avg     0.8085    0.8045    0.8039      8245



In [89]:
# CountVec + CNB
X_train, X_test, y_train, y_test = train_test_split(countvec_ngram_features, df['sentiment'], 
                                                    test_size=0.3, random_state=1)
model = ComplementNB()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, digits=4, target_names=list(map(str, list(y_test.unique())))))

              precision    recall  f1-score   support

    negative     0.7675    0.7217    0.7439      2282
     neutral     0.7676    0.8337    0.7993      3392
    positive     0.8410    0.7900    0.8147      2571

    accuracy                         0.7891      8245
   macro avg     0.7920    0.7818    0.7860      8245
weighted avg     0.7905    0.7891    0.7888      8245



## Как видно из результатов, лучшую точность показал TFIDF + CNB